#ECE 225 Project Model Prediction

In [2]:
import numpy as np 
import pandas as pd

df = pd.read_csv('data.csv')

df.drop(['id', 'name', 'artists', 'release_date'], axis = 1, inplace = True)

def quantize(x):
  if x<=25:
    return 0
  elif 25<x<=50:
    return 1
  elif 50<x<=75:
    return 2
  else:
    return 3

df['popularity'] = df["popularity"].apply(quantize)

print(df.shape)

#Data Cleaning 
Null_values = df.isnull().sum()
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)
print(df.shape) #Looking at the change in shape, looks like there were some missing values and duplicates

#We split the y values from the data frame
y = df.loc[:,"popularity"]
# y = y_column.values
df = df.drop(labels="popularity", axis=1) #delete y column 
print(y.shape)
print(df.columns)




(170653, 15)
(168742, 15)
(168742,)
Index(['valence', 'year', 'acousticness', 'danceability', 'duration_ms',
       'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'speechiness', 'tempo'],
      dtype='object')


In [3]:
df


,valence,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,speechiness,tempo
0,0.0594,1921,0.98200,0.279,831667,0.211,0,0.878000,10,0.6650,-20.096,1,0.0366,80.954
1,0.9630,1921,0.73200,0.819,180533,0.341,0,0.000000,7,0.1600,-12.441,1,0.4150,60.936
2,0.0394,1921,0.96100,0.328,500062,0.166,0,0.913000,3,0.1010,-14.850,1,0.0339,110.339
3,0.1650,1921,0.96700,0.275,210000,0.309,0,0.000028,5,0.3810,-9.316,1,0.0354,100.109
4,0.2530,1921,0.95700,0.418,166693,0.193,0,0.000002,3,0.2290,-10.096,1,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170648,0.6080,2020,0.08460,0.786,301714,0.808,0,0.000289,7,0.0822,-3.702,1,0.0881,105.029
170649,0.7340,2020,0.20600,0.717,150654,0.753,0,0.000000,7,0.1010,-6.020,1,0.0605,137.936
170650,0.6370,2020,0.10100,0.634,211280,0.858,0,0.000009,4,0.2580,-2.226,0,0.0809,91.688
170651,0.1950,2020,0.00998,0.671,337147,0.623,1,0.000008,2,0.6430,-7.161,1,0.3080,75.055


In [4]:
y

0         0
1         0
2         0
3         0
4         0
         ..
170648    2
170649    2
170650    3
170651    2
170652    2
Name: popularity, Length: 168742, dtype: int64

#Train / Test Split



In [6]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, train_size=0.80, stratify=y)
print('Frequency of each class with just random sampling in Y Train is ', Counter(y_train))
print('Frequency of each class with just random sampling in Y Train is ', Counter(y_test))

Frequency of each class with just random sampling in Y Train is  Counter({1: 54950, 0: 51293, 2: 27374, 3: 1376})
Frequency of each class with just random sampling in Y Train is  Counter({1: 13738, 0: 12824, 2: 6843, 3: 344})


#Model Analysis

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set_style('white')  
import warnings

warnings.filterwarnings('ignore')

# Initializing Classifiers
clf1 = LogisticRegression(multi_class='multinomial',
                          solver='newton-cg',
                          random_state=23225)
clf2 = KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)
clf3 = RandomForestClassifier()



# Building the pipelines
pipe1 = Pipeline([('std', StandardScaler()),
                  ('classifier', clf1)])

pipe2 = Pipeline([('std', StandardScaler()),
                  ('classifier', clf2)])


pipe3 = Pipeline([('classifier', clf3)]) 


#Choosing k between 1 and 1000, 26 values
k_flt = np.logspace(0.1, 3, num = 26)
k_list = []
for num in k_flt:
  k_list.append(int(num))

# Setting up the parameter grids
param_grid1 = [{'classifier__penalty': ['none','l2'],
                'classifier__C': np.power(10., np.arange(-4, 4))}] #Varying C

param_grid2 = [{'classifier__n_neighbors': k_list, #Varying K 
                'classifier__p': [1, 2]}]

param_grid3 = [{'classifier__n_estimators': [10, 100, 1000],
                 'classifier__max_features': [1, 2,4,6,8,12,16,20]}]



# Setting up multiple GridSearchCV objects, 1 for each algorithm
gridcvs = {}

for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3),
                            (pipe1, pipe2, pipe3),
                            ('Logistic', 'KNN', 'RF')):
  
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='accuracy',
                       n_jobs=1,
                       cv=2, # just 2-fold inner loop, i.e. train/test
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv

In [ ]:
%%time 

cv_scores = {name: [] for name, gs_est in gridcvs.items()}

skfold = StratifiedKFold( n_splits=3, shuffle=True, random_state=1)

# The outer loop for algorithm selection
c = 1
for outer_train_idx, outer_valid_idx in skfold.split(X_train,y_train):
    for name, gs_est in sorted(gridcvs.items()):
        print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

        # The inner loop for hyperparameter tuning
        gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
        y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
        acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
        print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
              (gs_est.best_score_ * 100, acc * 100))
        cv_scores[name].append(acc)

    c += 1




outer fold 1/5 | tuning KNN      | inner ACC 73.74% | outer ACC 74.07%
outer fold 1/5 | tuning Logistic | inner ACC 76.07% | outer ACC 75.78%
outer fold 1/5 | tuning RF       | inner ACC 76.83% | outer ACC 76.56%
outer fold 2/5 | tuning KNN      | inner ACC 73.69% | outer ACC 74.45%
outer fold 2/5 | tuning Logistic | inner ACC 75.89% | outer ACC 76.24%
outer fold 2/5 | tuning RF       | inner ACC 76.77% | outer ACC 76.91%
outer fold 3/5 | tuning KNN      | inner ACC 73.70% | outer ACC 74.21%
outer fold 3/5 | tuning Logistic | inner ACC 76.03% | outer ACC 76.04%
outer fold 3/5 | tuning RF       | inner ACC 76.85% | outer ACC 76.85%
CPU times: user 45min 23s, sys: 2min 16s, total: 47min 40s
Wall time: 45min 28s


In [ ]:
# Looking at the results
for name in cv_scores:
    print('%-8s | outer CV acc. %.2f%% +\- %.3f' % (
          name, 100 * np.mean(cv_scores[name]), 100 * np.std(cv_scores[name])))
print()
for name in cv_scores:
    print('{} best parameters'.format(name), gridcvs[name].best_params_)

Logistic | outer CV acc. 76.02% +\- 0.187
KNN      | outer CV acc. 74.24% +\- 0.153
RF       | outer CV acc. 76.77% +\- 0.153

Logistic best parameters {'classifier__C': 1.0, 'classifier__penalty': 'l2'}
KNN best parameters {'classifier__n_neighbors': 50, 'classifier__p': 1}
RF best parameters {'classifier__max_features': 2, 'classifier__n_estimators': 100}


In [ ]:
# Fitting a model to the whole training set
# using the "best" algorithm
best_algorithim = gridcvs['KNN']

best_algo.fit(X_train, y_train)
train_acc = accuracy_score(y_true=y_train, y_pred=best_algorithim.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algorithim.predict(X_test))

print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algorithim.best_score_))
print('Best Parameters: %s' % gridcvs['KNN'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

Accuracy 74.03% (average over CV test folds)
Best Parameters: {'classifier__n_neighbors': 50, 'classifier__p': 1}
Training Accuracy: 75.52%
Test Accuracy: 74.31%
